In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from statsmodels.tsa.arima.model import ARIMA
import geopandas as gpd

# Load and clean data
pas_data = pd.read_csv('../data/pas_boroughs.csv')
outcomes_data = pd.read_csv('../data/metropolitan-outcomes-merged.csv')
# stop_search_data = pd.read_csv('../data/metropolitan-stop-and-search-merged.csv')
# street_data = pd.read_csv('../data/metropolitan-street-merged.csv')

# Aggregating and creating crime rates
outcomes_data['date'] = pd.to_datetime(outcomes_data['Month'])
outcomes_data['year'] = outcomes_data['date'].dt.year
outcomes_data['quarter'] = outcomes_data['date'].dt.quarter

In [ ]:

# Load the CSV file
arrests = pd.read_csv("../data/Custody_Arrests_2019_2024.csv", delimiter=';')

# Display the information (first few rows) from the CSV file
print("Information from Custody_Arrests_2019_2024.csv:")
print(arrests.info())

# Find the number of rows in the original DataFrame 'arrests' where Age Group is neither 'Adult' nor 'Youth'
other_age_rows = arrests[~arrests['Age Group'].isin(['Adult', 'Youth'])]

# Get the number of rows
num_other_age_rows = other_age_rows.shape[0]

print("Number of rows with Age Group other than 'Adult' or 'Youth':", num_other_age_rows)

# Filter the DataFrame for 'Adult' age group
adult_rows = arrests[arrests['Age Group'] == 'Adult']

# Get the count of adult rows
num_adult_rows = adult_rows.shape[0]

# Filter the DataFrame for 'Youth' age group
youth_rows = arrests[arrests['Age Group'] == 'Youth']

# Get the count of youth rows
num_youth_rows = youth_rows.shape[0]

print("Number of Adult arrests:", num_adult_rows)
print("Number of Youth arrests:", num_youth_rows)

# Proportion of Youth arrests
total_arrests = len(arrests)
proportion_youth_arrests = num_youth_rows / total_arrests

print("Proportion of Youth arrests:", proportion_youth_arrests)

# Plotting proportion of Youth arrests
labels = ['Youth Arrests', 'Other Ages Arrests']
sizes = [num_youth_rows, total_arrests - num_youth_rows]
colors = ['orange', 'lightblue']
explode = (0.1, 0)  # explode the 1st slice

plt.figure(figsize=(8, 8))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%',
        shadow=True, startangle=140)
plt.title('Proportion of Youth Arrests')
plt.show()

# Filter out only for Youth arrests
youth_arrests = arrests[arrests['Age Group'] == 'Youth'].copy()

# Extract year and month
youth_arrests['Arrest Year'] = youth_arrests['Arrest Year'].astype(int)
youth_arrests['Arrest Month'] = youth_arrests['Arrest Month'].astype(int)

# Define quarter
youth_arrests['quarter'] = pd.cut(
    youth_arrests['Arrest Month'],
    bins=[0, 3, 6, 9, 12],
    labels=[1, 2, 3, 4],
    right=True
)

# Filter for the time span: quarter 1 of 2021 until quarter 3 of 2023
filtered_youth_arrests = youth_arrests[
    (youth_arrests['Arrest Year'] >= 2021) &
    (youth_arrests['Arrest Year'] <= 2023) &
    ~((youth_arrests['Arrest Year'] == 2023) & (youth_arrests['quarter'] > 3))
]

# Aggregate over quarter and year
aggregated_youth_arrests = filtered_youth_arrests.groupby(['Arrest Year', 'quarter']).size().reset_index(name='count')

print("Aggregated Youth Arrests:")
print(aggregated_youth_arrests)

# Plotting aggregated youth arrests by year and quarter
plt.figure(figsize=(10, 6))
sns.barplot(data=aggregated_youth_arrests, x='Arrest Year', y='count', hue='quarter')
plt.title('Aggregated Youth Arrests by Year and Quarter')
plt.xlabel('Year')
plt.ylabel('Number of Arrests')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

# Distribution of youth arrests for each 'First Arrest Offence'
youth_offence_distribution = youth_arrests['First Arrest Offnece'].value_counts()

print("Distribution of Youth Arrests by First Arrest Offence:")
print(youth_offence_distribution)

import matplotlib.pyplot as plt

# Plotting distribution of youth arrests by 'First Arrest Offence' with increased text size
plt.figure(figsize=(10, 8))
youth_offence_distribution.plot(kind='bar', color='orange')
plt.title('Distribution of Youth Arrests by First Arrest Offence', fontsize=16)
plt.xlabel('First Arrest Offence', fontsize=14)
plt.ylabel('Number of Arrests', fontsize=14)
plt.xticks(rotation=90, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Create a pivot table for youth arrests by quarter and "First Arrest Offnece"
pivot_table = pd.pivot_table(
    filtered_youth_arrests,
    index='First Arrest Offnece',
    columns=['Arrest Year', 'quarter'],
    aggfunc='size',
    fill_value=0
)

# Add a 'Total' column for each quarter
pivot_table['Total'] = pivot_table.sum(axis=1)

# Add a 'Total' row for each quarter
pivot_table.loc['Total'] = pivot_table.sum()

# Print the pivot table
print(pivot_table)

# Plot the pivot table
plt.figure(figsize=(15, 10))
sns.heatmap(pivot_table, cmap="plasma", annot=True, fmt="d", linewidths=.5)
plt.title('Youth Arrests per Quarter by First Arrest Offence')
plt.xlabel('Year, Quarter')
plt.ylabel('First Arrest Offence')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Print distribution of 'First Arrest Offnece' for youth arrests per quarter
offence_distribution_per_quarter = filtered_youth_arrests.groupby(['Arrest Year', 'quarter', 'First Arrest Offnece']).size().unstack(fill_value=0)
print("Distribution of Youth Arrests by First Arrest Offnece per Quarter:")
print(offence_distribution_per_quarter)

# Plot distribution
offence_distribution_per_quarter.T.plot(kind='bar', stacked=True, figsize=(14, 8), colormap='viridis')
plt.title('Distribution of Youth Arrests by First Arrest Offence per Quarter')
plt.xlabel('First Arrest Offence')
plt.ylabel('Number of Arrests')
plt.legend(title='Year & Quarter', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=90)
plt.tight_layout()
plt.grid(axis='y')
plt.show()

import seaborn as sns

# Create a pivot table for the heatmap
pivot_table = filtered_youth_arrests.pivot_table(
    index='First Arrest Offnece', 
    columns=['Arrest Year', 'quarter'], 
    aggfunc='size', 
    fill_value=0
)

# Plot the heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(pivot_table, cmap="plasma", annot=True, fmt="d", linewidths=.5)
plt.title('Heatmap of Youth Arrests by First Arrest Offence per Quarter')
plt.xlabel('Year and Quarter')
plt.ylabel('First Arrest Offence')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Melt the pas_data DataFrame to make it suitable for seaborn
melted_data = pas_data.melt(id_vars=['borough', 'quarter'], 
                            value_vars=[col for col in pas_data.columns if 'arrests' in col],
                            var_name='Offence_Quarter', 
                            value_name='Arrests')

# Extract 'Quarter' and 'Offence' from 'Offence_Quarter' column
melted_data[['Quarter', 'Offence']] = melted_data['Offence_Quarter'].str.extract(r'Q(\d)_(.*)_arrests')

# Convert Quarter to int
melted_data['Quarter'] = melted_data['Quarter'].astype(int)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting a heatmap for a clearer view of distribution per quarter
plt.figure(figsize=(65, 45))
pivot_heatmap = melted_data.pivot_table(index='borough', columns=['Quarter', 'Offence'], values='Arrests', aggfunc='sum', fill_value=0)

# Create the heatmap with larger font sizes
ax = sns.heatmap(pivot_heatmap, cmap='coolwarm', annot=True, fmt='g',
                 annot_kws={'size': 20})  # Adjust this to set annotation size

# Set title and axis labels with larger font sizes
plt.title('Heatmap of Youth Arrests per Quarter by First Arrest Offence and Borough', fontsize=30)
plt.xlabel('Quarter, Offence', fontsize=25)
plt.ylabel('Borough', fontsize=25)

# Set tick labels with larger font sizes
plt.xticks(rotation=45, fontsize=20)
plt.yticks(fontsize=20)

# Set color bar label size separately
cbar = ax.collections[0].colorbar
cbar.set_label('Number of Arrests', fontsize=25)
cbar.ax.tick_params(labelsize=20)

plt.tight_layout()
plt.show()

In [ ]:
# Part 1.b split the data into 2 parts for faster running on 2 computers

import pandas as pd

# Split the data into two equal parts
half_index = len(filtered_sas_11q) // 2
part1 = filtered_sas_11q.iloc[:half_index]
part2 = filtered_sas_11q.iloc[half_index:]

# Save both parts as CSV files
part1.to_csv('../data/filtered_sas_11q_part1.csv', index=False)
part2.to_csv('../data/filtered_sas_11q_part2.csv', index=False)

print(f"Part 1 saved with {len(part1)} rows")
print(f"Part 2 saved with {len(part2)} rows")

In [ ]:
# Part 2'

import time
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

boroughs = [
    "Barking and Dagenham",
    "Barnet",
    "Bexley",
    "Brent",
    "Bromley",
    "Camden",
    "Croydon",
    "Ealing",
    "Enfield",
    "Greenwich",
    "Hackney",
    "Hammersmith and Fulham",
    "Haringey",
    "Harrow",
    "Havering",
    "Hillingdon",
    "Hounslow",
    "Islington",
    "Kensington and Chelsea",
    "Kingston upon Thames",
    "Lambeth",
    "Lewisham",
    "Merton",
    "Newham",
    "Redbridge",
    "Richmond upon Thames",
    "Southwark",
    "Sutton",
    "Tower Hamlets",
    "Waltham Forest",
    "Wandsworth",
    "Westminster"
]
filtered_sas_11q_part1 = pd.read_csv('../data/filtered_sas_11q_part1.csv')

geolocator = Nominatim(user_agent="eda_geolocator")
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

def get_borough(lat, lon, row_index):
    if pd.notna(lat) and pd.notna(lon):
        start_time = time.time()  # Start time measurement
        location = geocode((lat, lon), language='en')
        end_time = time.time()  # End time measurement
        elapsed_time = end_time - start_time  # Calculate elapsed time
        print(f"Geolocation for row {row_index} took {elapsed_time} seconds.")
        if location:
            address = location.address
            # Split the address to find the borough name, adjust the split as necessary
            for b in boroughs:
                if b in address:
                    return b
    return None

# This code was used to add a Borough column that based on Latitude and Longitude

filtered_sas_11q_part1['Borough'] = filtered_sas_11q_part1.apply(
   lambda row: get_borough(row['Latitude'], row['Longitude'], row.name),  # Pass row index to function
   axis=1
)

filtered_sas_11q_part1['Borough'] = filtered_sas_11q_part1['Borough'].fillna("Unknown")

filtered_sas_11q_part1.to_csv('filtered_sas_11q_part1.csv', index=False)

In [ ]:
pas1 = pd.read_csv('../data/filtered_sas_part1.csv')
pas2 = pd.read_csv('../data/filtered_sas_11q_part2.csv')
print(pas1.info())
print(pas2.info())

In [ ]:
# Merge the DataFrames on the common column(s)
# merged_filtered_sas = pd.concat([pas1, pas2], ignore_index=True)  

# Display the first few rows of the merged DataFrame
print(merged_filtered_sas)

# Save the merged DataFrame to a CSV file
# merged_filtered_sas.to_csv('../data/merged_filtered_sas.csv', index=False)

In [ ]:
outcomes = pd.read_csv('../data/metropolitan-outcomes-merged.csv')
print(outcomes.info())
print(outcomes)

# Filter out rows where 'LSOA name' is missing and create a copy to avoid SettingWithCopyWarning
filtered_outcomes = outcomes.dropna(subset=['LSOA name']).copy()

# Use .loc to assign the modified 'LSOA name' column
filtered_outcomes.loc[:, 'LSOA name'] = filtered_outcomes['LSOA name'].apply(lambda x: x.split(' ')[0])

filtered_outcomes.rename(columns={'LSOA name': 'Borough'}, inplace=True)

# Display the info and first few rows to verify
print(filtered_outcomes.info())
print(filtered_outcomes.head())

# Save the filtered and modified DataFrame to a new CSV file
filtered_outcomes.to_csv('../data/filtered_outcomes.csv', index=False)

In [ ]:
import pandas as pd

# Load the filtered outcomes data
outcomes = pd.read_csv('../data/filtered_outcomes.csv')

# Define the list of boroughs to filter
boroughs = [
    "Barking and Dagenham",
    "Barking",
    "Barnet",
    "Bexley",
    "Brent",
    "Bromley",
    "Camden",
    "Croydon",
    "Ealing",
    "Enfield",
    "Greenwich",
    "Hackney",
    "Hammersmith and Fulham",
    "Hammersmith",
    "Haringey",
    "Harrow",
    "Havering",
    "Hillingdon",
    "Hounslow",
    "Islington",
    "Kensington and Chelsea",
    "Kensington",
    "Chelsea",
    "Kennington",
    "Kingston upon Thames",
    "Kingston",
    "Lambeth",
    "Lewisham",
    "Merton",
    "Newham",
    "Redbridge",
    "Richmond upon Thames",
    "Richmond",
    "Southwark",
    "Sutton",
    "Tower Hamlets",
    "Tower",
    "Waltham Forest",
    "Waltham",
    "Wandsworth",
    "Westminster"
]

# Filter the dataframe to include only the specified boroughs
filtered_outcomes = outcomes[outcomes['Borough'].isin(boroughs)]

# Print the counts for each of these boroughs
borough_counts = filtered_outcomes['Borough'].value_counts()
print(borough_counts.head(35))

In [ ]:
# Load the filtered outcomes data
outcomes = filtered_outcomes

# Define the mappings for short names to full borough names
borough_replacements = {
    "Barking": "Barking and Dagenham",
    "Hammersmith": "Hammersmith and Fulham",
    "Kensington": "Kensington and Chelsea",
    "Kingston": "Kingston upon Thames",
    "Richmond": "Richmond upon Thames",
    "Tower": "Tower Hamlets",
    "Waltham": "Waltham Forest"
}

# Replace the short borough names with the full names
outcomes['Borough'] = outcomes['Borough'].replace(borough_replacements)

# Print the counts for each of these boroughs
borough_counts = outcomes['Borough'].value_counts()
print(borough_counts)

In [ ]:
# Ensure 'Month' is in datetime format
outcomes['Month'] = pd.to_datetime(outcomes['Month'])

# Extract year and quarter
outcomes['Year'] = outcomes['Month'].dt.year
outcomes['Quarter'] = outcomes['Month'].dt.quarter

# Aggregate by Borough, Year, and Quarter
aggregated_outcomes = outcomes.groupby(['Borough', 'Year', 'Quarter']).size().reset_index(name='Count')

# Print the resulting aggregated DataFrame
print(aggregated_outcomes)

In [ ]:

# Convert 'Year' and 'Quarter' columns to match the data type of the other DataFrame
aggregated_outcomes['Year'] = aggregated_outcomes['Year'].astype(str)
aggregated_outcomes['Quarter'] = aggregated_outcomes['Quarter'].astype(float)

# Sort the DataFrame by Borough, Year, and Quarter
aggregated_outcomes_sorted = aggregated_outcomes.sort_values(by=['Borough', 'Year', 'Quarter'])


In [ ]:
import pandas as pd

# Create a DataFrame to hold the new rows
new_rows = []

# Iterate through each unique Borough and Year in the DataFrame
for borough in aggregated_outcomes_sorted['Borough'].unique():
    for year in aggregated_outcomes_sorted['Year'].unique():
        # Filter the DataFrame for the specific Borough and Year
        df_filtered = aggregated_outcomes_sorted[(aggregated_outcomes_sorted['Borough'] == borough) & (aggregated_outcomes_sorted['Year'] == year)]
        
        # Check if the filtered DataFrame has data for quarters 2, 3, and 4
        if len(df_filtered) == 3:
            # Calculate the average count for quarters 2, 3, and 4
            avg_count = df_filtered['Count'].mean()
            
            # Append a new row for Quarter 1
            new_rows.append({'Borough': borough, 'Year': year, 'Quarter': 1.0, 'Count': avg_count})

# Convert the new rows to a DataFrame and append to the original DataFrame
new_rows_df = pd.DataFrame(new_rows)
aggregated_outcomes_updated = pd.concat([aggregated_outcomes_sorted, new_rows_df], ignore_index=True)

# Sort the final DataFrame by Borough, Year, and Quarter
aggregated_outcomes_final = aggregated_outcomes_final.sort_values(by=['Borough', 'Year', 'Quarter']).reset_index(drop=True)

# Print the final DataFrame info to verify the row count
print(aggregated_outcomes_final.info())
print(aggregated_outcomes_final)

In [ ]:
# Define the unique boroughs
pas_final = pd.read_csv('../data/pas_final.csv')
# Iterate through the DataFrame
new_rows = []

for index, row in pas_final.iterrows():
    if row['Year'] == 2023 and row['Quarter'] == 4.0:
        # Save the values
        new_row = row.copy()
        new_row['Year'] = 2021
        new_row['Quarter'] = 1.0
        new_rows.append(new_row)
        # Drop the current row
        pas_final.drop(index, inplace=True)

# Add the new rows for Year 2021 and Quarter 1
pas_final = pd.concat([pas_final, pd.DataFrame(new_rows)], ignore_index=True)